In [2]:
# CONECTAR CON DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import sys
path ='/content/drive/MyDrive/cod/LEA3_Marketing'
os.chdir(path) ## volver la carpeta de repositorio directorio de trabajo
sys.path.append(path) ## agregarla al path, para leer archivos propios como paquetes

In [4]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
#### conectar_base_de_Datos
#!pip install ipywidgets

# CREAR CONEXIÓN CON LA BASE DE DATOS db_movies
con = sql.connect('data/db_movies')

# CREAR EL CURSOR
cur = con.cursor() ## se crea el cursor, que es el otro tipo de conexión para ejecutar las consultas

In [11]:
# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas
pd.set_option('display.width', 1000)  # Aumenta el ancho de la línea

In [5]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('ratings',), ('movies',)]


## TOP 10 Peliculas con mejores calificaciones y mas calificadas
###  Se usa pesos para dar a los mas calificados mejor puntuacion asi como a los mejor calificados mas puntuacion, entonces una pelicula con muchas calificaciones y bien calificadas tendra gran oportunidad de hacer parte de este top 10 de popularidad

In [ ]:
# Calcular el promedio de calificación (C) y el percentil 75 de cantidad de calificaciones (m)

# Primero, el conteo de rantings
rating_counts_df = pd.read_sql("""
    SELECT COUNT(*) AS rating_count
    FROM ratings
    GROUP BY movieId
""", con)

# Calificacion promedio / rating promedio
avg_rating_df = pd.read_sql("""
    SELECT AVG(rating) AS C
    FROM ratings
""", con)

# Extraccion de calificación promedio
C = avg_rating_df['C'].iloc[0]

# calculo del percentil 75.
m = rating_counts_df['rating_count'].quantile(0.75)

print(f"Rating promedio (C): {C}")
print(f"percentil 75 del conteo de calificaciones (m): {m}")

Average Rating (C): 3.501556983616962
75th Percentile of Rating Counts (m): 9.0


In [12]:
## se prepara el query donde  calcula la calificacion ponderada dando uso a los productos mas calificados y con mejor calificacion.
query = f"""
    SELECT m.title,
           AVG(r.rating) AS avg_rating,
           COUNT(*) AS rating_count,
           -- Calcular rating con peso por vista y calificación
           ((COUNT(*) / (COUNT(*) + {m})) * AVG(r.rating) +
           ({m} / (COUNT(*) + {m})) * {C}) AS weighted_rating
    FROM ratings r
    JOIN movies m ON r.movieId = m.movieId
    WHERE r.rating > 0
    GROUP BY m.title
    HAVING rating_count > 30
    ORDER BY weighted_rating DESC
    LIMIT 10
"""
df = pd.read_sql(query, con)



print(df)

                                               title  avg_rating  rating_count  weighted_rating
0                   Shawshank Redemption, The (1994)    4.429022           317         4.403417
1                              Godfather, The (1972)    4.289062           192         4.253801
2                                  Fight Club (1999)    4.272936           218         4.242352
3                     Godfather: Part II, The (1974)    4.259690           129         4.210246
4                         Usual Suspects, The (1995)    4.237745           204         4.206639
5          Star Wars: Episode IV - A New Hope (1977)    4.231076           251         4.205823
6  Dr. Strangelove or: How I Learned to Stop Worr...    4.268041            97         4.202962
7                                  Goodfellas (1990)    4.250000           126         4.200104
8                            Schindler's List (1993)    4.225000           220         4.196568
9                            Dark Knight

In [ ]:
## Pendiente de revisar ya sea mejor almacenando la consulta en un dataframe para manipular

#### los mejores calificados por año publicacion ###
pd.read_sql("""
  SELECT
      SUBSTR(m.title, LENGTH(m.title) - 5, 6) AS movie_year,  -- Extrae las últimas 4 cifras (año)
      SUBSTR(m.title, 1, LENGTH(m.title) - 7) AS movie_name,  -- Extrae el nombre quitando el año
      MAX(r.rating) AS best_rating
  FROM movies m
  JOIN ratings r ON m.movieId = r.movieId
  GROUP BY movie_year
  ORDER BY movie_year;"""
    ,con)

In [ ]:
movies=pd.read_sql("""select * from movies""", con)


In [ ]:
sep=movies['title'].str.split('(')

year=sep.str[1].str.replace(')','')
title=sep.str[0]

In [ ]:
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)$')
movies['title_clean'] = movies['title'].str.replace(r'\s*\(\d{4}\)$', '', regex=True)

In [ ]:
movies.drop(columns=['title'], inplace=True)
movies.rename(columns={'title_clean': 'title'}, inplace=True)

In [ ]:
movies

In [ ]:
movies=movies[movies.columns[1:].tolist() + [movies.columns[0]]]
movies.drop(columns=['genres'], inplace=True)
movies

In [ ]:
#######################################################################
######## 2.1 Sistema de recomendación basado en contenido un solo producto - Manual ########
#######################################################################

movies_2=pd.read_sql('select * from movies_final', con )

movies_2.info()
movies_2['año_estreno']=movies_2.año_estreno.astype('int')
movies_2.info()

##### escalar para que año esté en el mismo rango ###

sc=MinMaxScaler()
movies_2[["year_sc"]]=sc.fit_transform(movies_2[['year_pub']])



## eliminar filas que no se van a utilizar ###

movies_2_dum1=movies_2.drop(columns=['isbn','i_url','year_pub','book_title']) ## ingresar las columnas a quitar ###

#### convertir a dummies

movies_2_dum1['book_author'].nunique()
movies_2_dum1['publisher'].nunique()

col_dum=['book_author','publisher']
movies_2_dum2=pd.get_dummies(movies_2_dum1,columns=col_dum)
movies_2_dum2.shape

joblib.dump(movies_2_dum2,"salidas\\movies_2_dum2.joblib") ### para utilizar en segundos modelos


